In [67]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import ast

df = pd.read_csv('/kaggle/input/books-dataset-goodreadsmay-2024/Book_Details.csv',index_col = 'book_title')
df.head()

,Unnamed: 0,book_id,cover_image_uri,book_details,format,publication_info,authorlink,author,num_pages,genres,num_ratings,num_reviews,average_rating,rating_distribution
book_title,,,,,,,,,,,,,,
Harry Potter and the Half-Blood Prince,0,1,https://images-na.ssl-images-amazon.com/images...,"It is the middle of the summer, but there is a...","['652 pages, Paperback']","['First published July 16, 2005']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['652'],"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",3292516,58398,4.58,"{'5': '2,244,154', '4': '775,028', '3': '219,8..."
Harry Potter and the Order of the Phoenix,1,2,https://images-na.ssl-images-amazon.com/images...,Harry Potter is about to start his fifth year ...,"['912 pages, Paperback']","['First published June 21, 2003']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['912'],"['Young Adult', 'Fiction', 'Magic', 'Childrens...",3401709,64300,4.50,"{'5': '2,178,760', '4': '856,178', '3': '293,2..."
Harry Potter and the Sorcerer's Stone,2,3,https://images-na.ssl-images-amazon.com/images...,Harry Potter has no idea how famous he is. Tha...,"['309 pages, Hardcover']","['First published June 26, 1997']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['309'],"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",10116247,163493,4.47,"{'5': '6,544,542', '4': '2,348,390', '3': '856..."
Harry Potter and the Prisoner of Azkaban,3,5,https://images-na.ssl-images-amazon.com/images...,"Harry Potter, along with his best friends, Ron...","['435 pages, Mass Market Paperback']","['First published July 8, 1999']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['435'],"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4215031,84959,4.58,"{'5': '2,892,322', '4': '970,190', '3': '287,7..."
Harry Potter and the Goblet of Fire,4,6,https://images-na.ssl-images-amazon.com/images...,It is the summer holidays and soon Harry Potte...,"['734 pages, Paperback']","['First published July 8, 2000']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['734'],"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",3718209,69961,4.57,"{'5': '2,500,070', '4': '899,496', '3': '259,7..."


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16225 entries, Harry Potter and the Half-Blood Prince to Shadow's Siege
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           16225 non-null  int64  
 1   book_id              16225 non-null  int64  
 2   cover_image_uri      16225 non-null  object 
 3   book_details         16177 non-null  object 
 4   format               16225 non-null  object 
 5   publication_info     16225 non-null  object 
 6   authorlink           16225 non-null  object 
 7   author               16225 non-null  object 
 8   num_pages            16225 non-null  object 
 9   genres               16225 non-null  object 
 10  num_ratings          16225 non-null  int64  
 11  num_reviews          16225 non-null  int64  
 12  average_rating       16225 non-null  float64
 13  rating_distribution  16225 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usa

In [69]:
df.describe()

,Unnamed: 0,book_id,num_ratings,num_reviews,average_rating
count,16225.000000,1.622500e+04,1.622500e+04,16225.000000,16225.000000
mean,8112.000000,1.038704e+07,8.578453e+04,5156.269584,4.002499
std,4683.898394,2.146809e+07,3.202513e+05,15776.459696,0.336102
min,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000
25%,4056.000000,9.403400e+04,2.950000e+03,205.000000,3.830000
50%,8112.000000,9.440730e+05,1.623200e+04,993.000000,4.030000
75%,12168.000000,1.128490e+07,5.594400e+04,3457.000000,4.210000
max,16224.000000,2.096076e+08,1.011625e+07,274695.000000,5.000000


In [70]:
df.shape

(16225, 14)

In [71]:
df.isna().sum()

Unnamed: 0              0
book_id                 0
cover_image_uri         0
book_details           48
format                  0
publication_info        0
authorlink              0
author                  0
num_pages               0
genres                  0
num_ratings             0
num_reviews             0
average_rating          0
rating_distribution     0
dtype: int64

In [72]:
df = df.dropna()
df['genres'] = df['genres'].str.strip('[]').str.split(',').apply(lambda x: [i.strip().strip("'") for i in x])

In [73]:
list_genres_5 = ['Fantasy', 'Young Adult', 'Classics', 'Romance', 'Historical Fiction']

for genre in list_genres_5:
    df[genre] = df['genres'].apply(lambda x: genre in x)

In [74]:
df = df.loc[df['genres'].apply(lambda x: 'Fiction' in x),['book_details',*list_genres_5]]
df.head()

,book_details,Fantasy,Young Adult,Classics,Romance,Historical Fiction
book_title,,,,,,
Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...",True,True,False,False,False
Harry Potter and the Order of the Phoenix,Harry Potter is about to start his fifth year ...,False,True,False,False,False
Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...,True,True,False,False,False
Harry Potter and the Prisoner of Azkaban,"Harry Potter, along with his best friends, Ron...",True,True,False,False,False
Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...,True,True,False,False,False


In [75]:
df.shape

(11440, 6)

In [77]:
df.to_csv('preprocessed_data.csv')